In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
#from tensorflow.keras.utils import to_categorical ## specific  one I wanna use
from tensorflow.keras import utils
#from tensorflow.keras.layers import Input, Dense, Dropout  # Specific ones I wanna use, can just import whole module
from tensorflow.keras import layers 
from tensorflow.keras.models import Model
import h5py
#import tables
import matplotlib.pyplot as plt
#import tensorflow as tf
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.utils.class_weight import compute_class_weight


In [2]:
import tensorflow as tf
gpu_available = tf.test.is_gpu_available()
gpu_available

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [2]:
x=1
y=3
x+y



4

In [3]:
# Import data from all 4 collision types.

input_file_1 = "/scratch/ys20884/ml_vars/TTToHadronic/df_ml_inputs.hd5"

with h5py.File(input_file_1, "r") as f:
    #print("Contents of the file:")
    for key in f.keys():
        print(key)

df1= pd.read_hdf(input_file_1, key="df")


input_file2 = "/scratch/ys20884/ml_vars/TTToSemiLeptonic/df_ml_inputs.hd5"

with h5py.File(input_file2, "r") as f:
    #print("Contents of the file:")
    for key in f.keys():
        print(key)

df2 = pd.read_hdf(input_file2, key="df")
print("Dataframe columns:")
print(df2.columns)

input_file3 = "/scratch/ys20884/ml_vars/TTTo2L2Nu/df_ml_inputs.hd5"

with h5py.File(input_file3, "r") as f:
    #print("Contents of the file:")
    for key in f.keys():
        print(key)

df3 = pd.read_hdf(input_file3, key="df")

input_file4 = "/scratch/ys20884/ml_vars/ttH125/df_ml_inputs.hd5"

with h5py.File(input_file4, "r") as f:
    #print("Contents of the file:")
    for key in f.keys():
        print(key)

df4 = pd.read_hdf(input_file4, key="df")

print(len(df1))
print(len(df2))
print(len(df3))
print(len(df4))

df
df
Dataframe columns:
Index(['dataset', 'entry', 'BiasedDPhi', 'DiJet_mass', 'HT',
       'InputMet_InputJet_mindPhi', 'InputMet_phi', 'InputMet_pt', 'MHT_phi',
       'MHT_pt', 'MinChi', 'MinOmegaHat', 'MinOmegaTilde',
       'boostedObject_area', 'boostedObject_eta', 'boostedObject_mass',
       'boostedObject_phi', 'boostedObject_pt', 'boostedTop_area',
       'boostedTop_eta', 'boostedTop_mass', 'boostedTop_phi', 'boostedTop_pt',
       'boostedTop_tag', 'boostedV_area', 'boostedV_eta', 'boostedV_mass',
       'boostedV_phi', 'boostedV_pt', 'boostedV_tag', 'cleanedBJet_eta',
       'cleanedBJet_phi', 'cleanedBJet_pt', 'cleanedJet_area',
       'cleanedJet_btagDeepB', 'cleanedJet_chHEF', 'cleanedJet_eta',
       'cleanedJet_mass', 'cleanedJet_neHEF', 'cleanedJet_phi',
       'cleanedJet_pt', 'hashed_filename', 'ncleanedBJet', 'ncleanedJet',
       'weight_nominal', 'xs_weight'],
      dtype='object')
df
df
12185
651387
39666
177858


In [22]:
# make one big dataframe, full data set long to practice with
df_total = df2.append(df1)
df_total = df_total.append(df3)
df_total = df_total.append(df4)
print(len(df_total))   

# Background is 0
# Signal is 1
df_process = df_total['dataset']
df_total['dataset'] = df_total['dataset'].replace({'ttH125': 1, 'TTToSemiLeptonic': 0, 'TTToHadronic': 0, 'TTTo2L2Nu':0})
df_total.head()
# Have done test cases, is working fine.
print(len(df_total))   

df_total.columns

881096
881096


Index(['dataset', 'entry', 'BiasedDPhi', 'DiJet_mass', 'HT',
       'InputMet_InputJet_mindPhi', 'InputMet_phi', 'InputMet_pt', 'MHT_phi',
       'MHT_pt', 'MinChi', 'MinOmegaHat', 'MinOmegaTilde',
       'boostedObject_area', 'boostedObject_eta', 'boostedObject_mass',
       'boostedObject_phi', 'boostedObject_pt', 'boostedTop_area',
       'boostedTop_eta', 'boostedTop_mass', 'boostedTop_phi', 'boostedTop_pt',
       'boostedTop_tag', 'boostedV_area', 'boostedV_eta', 'boostedV_mass',
       'boostedV_phi', 'boostedV_pt', 'boostedV_tag', 'cleanedBJet_eta',
       'cleanedBJet_phi', 'cleanedBJet_pt', 'cleanedJet_area',
       'cleanedJet_btagDeepB', 'cleanedJet_chHEF', 'cleanedJet_eta',
       'cleanedJet_mass', 'cleanedJet_neHEF', 'cleanedJet_phi',
       'cleanedJet_pt', 'hashed_filename', 'ncleanedBJet', 'ncleanedJet',
       'weight_nominal', 'xs_weight'],
      dtype='object')

In [23]:
#df_weights = df_total['xs_weight']
df_total.drop(['entry', 'BiasedDPhi', 'hashed_filename', 'weight_nominal', 'xs_weight', 
              'InputMet_phi', 'MHT_phi', 'boostedObject_phi', 'boostedTop_phi', 'boostedV_phi',
              'cleanedBJet_phi','cleanedJet_phi'], axis=1, inplace=True)#


# <1%
#variables_permuter_drop = ['boostedObject_area', 'boostedObject_eta', 'boostedTop_eta', 'boostedTop_mass', 
#                          'boostedTop_pt', 'boostedTop_tag', 'boostedV_eta', 'boostedV_pt', 'cleanedBJet_eta',
#                          'cleanedJet_area', 'cleanedJet_chHEF']  # could be useful if add other types of data

# < 3%
# variables_permuter_drop = ['MinOmegaHat', 'boostedObject_area', 'boostedObject_eta',   # 3%
#                            'boostedObject_mass', 'boostedObject_pt', 'boostedTop_area',
#                            'boostedTop_eta', 'boostedTop_mass', 'boostedTop_pt', 'boostedTop_tag',
#                            'boostedV_area', 'boostedV_eta', 'boostedV_mass', 'boostedV_pt', 'boostedV_tag', 
#                            'cleanedBJet_eta', 'cleanedJet_area', 'cleanedJet_btagDeepB', 'cleanedJet_chHEF', 
#                            'cleanedJet_neHEF']

#df_total.drop(variables_permuter_drop, axis=1, inplace=True)#
df_total.columns            # Said BiasedDPhi looks wrong

# will drop label column too, as do this seperately in another one

Index(['dataset', 'DiJet_mass', 'HT', 'InputMet_InputJet_mindPhi',
       'InputMet_pt', 'MHT_pt', 'MinChi', 'MinOmegaHat', 'MinOmegaTilde',
       'boostedObject_area', 'boostedObject_eta', 'boostedObject_mass',
       'boostedObject_pt', 'boostedTop_area', 'boostedTop_eta',
       'boostedTop_mass', 'boostedTop_pt', 'boostedTop_tag', 'boostedV_area',
       'boostedV_eta', 'boostedV_mass', 'boostedV_pt', 'boostedV_tag',
       'cleanedBJet_eta', 'cleanedBJet_pt', 'cleanedJet_area',
       'cleanedJet_btagDeepB', 'cleanedJet_chHEF', 'cleanedJet_eta',
       'cleanedJet_mass', 'cleanedJet_neHEF', 'cleanedJet_pt', 'ncleanedBJet',
       'ncleanedJet'],
      dtype='object')

In [25]:
lst = []
for column in df_total:
    if df_total[column].dtype != 'object':   
        print(column, 'is an event based variable')
        lst.append(column)
    else:
        df_total.drop([column], axis=1, inplace=True)
print(len(lst), 'variables are left')    # -1 as - label
df_total.columns 

dataset is an event based variable
DiJet_mass is an event based variable
HT is an event based variable
InputMet_InputJet_mindPhi is an event based variable
InputMet_pt is an event based variable
MHT_pt is an event based variable
MinChi is an event based variable
MinOmegaHat is an event based variable
MinOmegaTilde is an event based variable
ncleanedBJet is an event based variable
ncleanedJet is an event based variable
11 variables are left


Index(['dataset', 'DiJet_mass', 'HT', 'InputMet_InputJet_mindPhi',
       'InputMet_pt', 'MHT_pt', 'MinChi', 'MinOmegaHat', 'MinOmegaTilde',
       'ncleanedBJet', 'ncleanedJet'],
      dtype='object')

In [27]:
df_total

,dataset,DiJet_mass,HT,InputMet_InputJet_mindPhi,InputMet_pt,MHT_pt,MinChi,MinOmegaHat,MinOmegaTilde,ncleanedBJet,ncleanedJet
0,0,363.519776,409.411560,0.248169,253.625,230.740250,0.084954,0.294236,0.023754,0,2
1,0,129.015341,466.076111,1.818359,308.250,339.473511,1.546004,1.134799,0.854612,1,5
2,0,274.769255,361.570465,0.668945,278.250,264.123840,0.621207,0.710851,0.103618,1,2
3,0,82.854809,359.750671,0.380127,218.750,227.014053,0.612625,0.612626,0.227535,0,3
4,0,82.354687,314.990326,1.387695,234.625,254.779617,1.441802,1.138652,0.881872,0,4
...,...,...,...,...,...,...,...,...,...,...,...
177853,1,353.306746,550.468018,0.340576,263.250,239.813583,0.811556,0.646585,0.334731,0,4
177854,1,206.457350,625.009277,0.468018,313.500,335.351562,0.991184,0.991184,0.872011,0,7
177855,1,511.200694,1002.960999,0.073547,245.875,249.904602,0.145308,0.145308,0.145308,1,8
177856,1,271.667337,525.788696,1.009766,210.500,226.901993,1.204100,0.954423,0.820283,1,5


In [7]:
# A function that makes arrays in rows singular values, max if multiple in array, 0 if empty array
def modify_array(arr):    
    if len(arr) == 0:
        return 0
    else:
    # takes leading jet    
        return max(arr)    #Need values as floats, not as arrays as NN did not accept arrays, needed as float
                            # If wanted arrays, put [] around returns, simple as that.

# e.g. df["column_name"] = df["column_name"].apply(modify_array)#
#e.g   df_total.boostedTop_area = df_total.boostedTop_area.apply(modify_array)

In [8]:
# This should have manipulated all data so is now fit for a ML model
for column in df_total:
    if df_total[column].dtype == 'object':
        df_total[column] = df_total[column].apply(modify_array)


In [28]:
with h5py.File('DF_event_based.h5', 'w') as f:
    f.create_dataset('dataset_name', data=df_total)

In [ ]:

# # Load the HDF5 file
# with h5py.File('DF_event_based.h5', 'r') as hf:
#     # Access the dataset by name and load it into a pandas dataframe
#     df_total = pd.DataFrame(f['dataset_name'][:])
